### Test BDD

In [ ]:
import os
import psycopg2
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine 

#### Création BDD

In [ ]:
mot_passe = os.environ.get('pg_psw')


In [ ]:
conn = psycopg2.connect(
   database="postgres", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

sql = '''CREATE database ProjetSPQR''';

cursor.execute(sql)
print("Base de données créée avec succès !")

conn.close()


In [ ]:
conn = psycopg2.connect(
   database="projetspqr", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

#### Création des tables

In [ ]:
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

In [ ]:
sql_creer_table_personne = """
    CREATE TABLE IF NOT EXISTS personne (
    id float,
    nom_fr text,
    praenomen text,
    nomen text,
    cognomen text,
    gens text,
    sexe text,
    date_naissance date,
    lieu_naissance text,
    date_mort date,
    lieu_mort text,
    id_pere float,
    id_mere float,
    PRIMARY KEY(id)
    );
"""

In [ ]:
sql_creer_table_activite = """
    CREATE TABLE IF NOT EXISTS activite (
    personne_id float,
    activite text,
    CONSTRAINT fk_personne
        FOREIGN KEY(personne_id)
            REFERENCES personne(id)
    );
"""

In [ ]:
sql_creer_table_poste = """
    CREATE TABLE IF NOT EXISTS poste (
    personne_id float,
    poste text,
    CONSTRAINT fk_personne
        FOREIGN KEY(personne_id)
            REFERENCES personne(id)
    );
"""

In [ ]:
sql_creer_table_mariage = """
    CREATE TABLE IF NOT EXISTS mariage (
    id serial,
    id_mari float,
    id_epouse float,
    PRIMARY KEY(id),
    CONSTRAINT fk_personne
        FOREIGN KEY(id_mari)
            REFERENCES personne(id),
        FOREIGN KEY(id_epouse)
            REFERENCES personne(id)
    );
"""

In [ ]:
sql_creer_table_oeuvre = """
    CREATE TABLE IF NOT EXISTS oeuvre (
    id integer,
    titre_fr text,
    titre_lat text,
    auteur_id float,
    genre text,
    PRIMARY KEY(id),
    CONSTRAINT fk_personne
        FOREIGN KEY(auteur_id)
            REFERENCES personne(id)
);
"""

In [ ]:
creer_table(conn, sql_creer_table_personne)
creer_table(conn, sql_creer_table_activite)
creer_table(conn, sql_creer_table_poste)
creer_table(conn, sql_creer_table_mariage)
creer_table(conn, sql_creer_table_oeuvre)

#### Dataframes

In [ ]:
df_personne = pd.read_table("query_personne.csv", delimiter =",") 
df_personne

In [ ]:
df_kingdom = pd.read_table("query_kingdom.csv", delimiter =",") 
df_early_republic = pd.read_table("query_early_republic.csv", delimiter =",") 
df_middle_republic = pd.read_table("query_middle_republic.csv", delimiter =",") 
df_late_republic = pd.read_table("query_late_republic.csv", delimiter =",") 
df_high_empire = pd.read_table("query_high_empire.csv", delimiter =",") 
df_low_empire = pd.read_table("query_low_empire.csv", delimiter =",")
df_spouses = pd.read_table("query_spouses.csv", delimiter =",") 

In [ ]:
df_personne = df_personne.append(df_kingdom, ignore_index=True)
df_personne = df_personne.append(df_early_republic, ignore_index=True)
df_personne = df_personne.append(df_middle_republic, ignore_index=True)
df_personne = df_personne.append(df_late_republic, ignore_index=True)
df_personne = df_personne.append(df_high_empire, ignore_index=True)
df_personne = df_personne.append(df_low_empire, ignore_index=True)


In [ ]:
df_personne.rename(columns={"item": "id",
                            "itemLabel": "nom_fr",
                            "praenomenLabel": "praenomen",
                            "nomenLabel": "nomen",
                            "cognomenLabel": "cognomen",
                            "gensLabel": "gens",
                            "sexeLabel": "sexe",
                            "pere": "id_pere",
                            "mere": "id_mere"},
                  inplace=True)

In [ ]:
print(df_personne.head())

In [ ]:
df_personne['id'] = df_personne.id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['lieu_naissance'] = df_personne.lieu_naissance.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['lieu_mort'] = df_personne.lieu_mort.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['id_pere'] = df_personne.id_pere.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['id_mere'] = df_personne.id_mere.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
df_personne

In [ ]:
df_personne.sort_values("id")

In [ ]:
df_personne.drop_duplicates(subset="id", keep="first", inplace=True)

In [ ]:
df_personne

In [ ]:
df_personne2 = df_personne.replace(to_replace ='t[0-9]+', value = np.nan, regex = True)
df_personne3 = df_personne2.replace(to_replace = '^Q[0-9]+', value = np.nan, regex = True)
df_personne3['date_naissance'] = df_personne3.date_naissance.str.replace('T00:00:00Z', '', regex=True)
df_personne3['date_mort'] = df_personne3.date_mort.str.replace('T00:00:00Z', '', regex=True)

In [ ]:
df_personne3

In [ ]:
df_personne4 = df_personne3.replace(r"^-([0-9]*-[0-9]*-[0-9]*)$", r"\1BC", regex = True)
df_personne4 = df_personne4.replace(r"^0000(-[0-9]*-[0-9]*)$", r"0001\1BC", regex = True)
df_personne4

In [ ]:
df_personne5 = df_personne4.where(pd.notnull(df_personne3), None)

In [ ]:
df_personne5

In [ ]:
personne_id_list = df_personne3['id'].tolist()


##### activite

In [ ]:
df_activite = pd.read_table("query_activite.csv", delimiter =",") 
print(df_activite.head()) 

In [ ]:
df_activite.rename(columns={"item": "personne_id",
                            "occupationLabel": "activite"},
                  inplace=True)

In [ ]:
df_activite['personne_id'] = df_activite.personne_id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
print(df_activite.head()) 

In [ ]:
df_activite2 = df_activite.replace(to_replace = '^t[0-9]+', value = np.nan, regex = True)
df_activite3 = df_activite2.replace(to_replace = '^Q[0-9]+', value = np.nan, regex = True)


In [ ]:
df_activite3

##### poste

In [ ]:
df_poste = pd.read_table("query_poste.csv", delimiter =",") 
print(df_poste.head()) 

In [ ]:
df_poste.rename(columns={"item": "personne_id",
                            "positionLabel": "poste"},
                  inplace=True)

In [ ]:
df_poste['personne_id'] = df_poste.personne_id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
print(df_poste.head()) 

In [ ]:
df_poste2 = df_poste.replace(to_replace ='t[0-9]+', value = np.nan, regex = True)
df_poste3 = df_poste2.replace(to_replace = '^Q[0-9]+', value = np.nan, regex = True)


In [ ]:
df_poste3

##### oeuvre

In [ ]:
df_oeuvre = pd.read_table("query_oeuvre.csv", delimiter =",") 
print(df_oeuvre.head()) 

In [ ]:
df_oeuvre.rename(columns={"oeuvre": "id",
                          "oeuvreLabel": "titre_fr",
                          "titre": "titre_lat",
                          "auteur": "auteur_id",
                          "genreLabel": "genre"},
                  inplace=True)

In [ ]:
for index, row in df_oeuvre.iterrows():
    e = df_oeuvre.at[index, 'id']
    e.replace('http://www.wikidata.org/entity/Q', '')

In [ ]:
print(df_oeuvre.head()) 

In [ ]:
df_oeuvre['id'] = df_oeuvre.id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_oeuvre['auteur_id'] = df_oeuvre.auteur_id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
df_oeuvre

In [ ]:
df_oeuvre.drop_duplicates(subset="id", keep="first", inplace=True)

In [ ]:
df_oeuvre

In [ ]:
df_oeuvre2 = df_oeuvre.replace(to_replace ='^t[0-9]+', value = np.nan, regex = True)
df_oeuvre3 = df_oeuvre2.replace(to_replace = '^Q[0-9]+', value = np.nan, regex = True)


In [ ]:
df_oeuvre3

##### mariage

In [ ]:
df_mariage = pd.read_table("query_mariage.csv", delimiter =",") 
print(df_mariage.head()) 

In [ ]:
df_mariage.rename(columns={"item": "id_mari",
                          "epouse": "id_epouse"},
                  inplace=True)

In [ ]:
print(df_mariage.head())

In [ ]:
df_mariage['id_mari'] = df_mariage.id_mari.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_mariage['id_epouse'] = df_mariage.id_epouse.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [ ]:
print(df_mariage.head()) 

In [ ]:
df_mariage2 = df_mariage.replace(to_replace ='^t[0-9]+', value = np.nan, regex = True)
df_mariage3 = df_mariage2.replace(to_replace = '^Q[0-9]+', value = np.nan, regex = True)
df_mariage3

In [ ]:
df_mariage3.dropna(subset = ["id_epouse"], inplace=True)
df_mariage3

In [ ]:
for e in df_mariage3['id_epouse']:
    if e not in personne_id_list:
        df_mariage3.drop(df_mariage3.loc[df_mariage3['id_epouse']==e].index, inplace=True)

In [ ]:
df_mariage3

In [ ]:
df_mariage4 = df_mariage3.reset_index(drop=True)
df_mariage4

In [ ]:
df_mariage4.insert(0, 'id', range(0, len(df_mariage4)))

df_mariage4

#### Insertion

In [ ]:
sql_insert_personne = """INSERT INTO personne (id, nom_fr, praenomen, nomen, cognomen, gens, sexe, date_naissance, lieu_naissance, date_mort, lieu_mort, id_pere, id_mere) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [ ]:
sql_insert_oeuvre = """INSERT INTO oeuvre (id, titre_fr, titre_lat, auteur_id, genre) VALUES (%s, %s, %s, %s, %s)"""

In [ ]:
sql_insert_poste = """INSERT INTO poste (personne_id, poste) VALUES (%s, %s)"""

In [ ]:
sql_insert_activite = """INSERT INTO activite (personne_id, activite) VALUES (%s, %s)"""

In [ ]:
sql_insert_mariage = """INSERT INTO mariage (id, id_mari, id_epouse) VALUES (%s, %s, %s)"""

In [ ]:
conn = psycopg2.connect(
   database="projetspqr", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

for index, row in df_personne5.iterrows():
    cursor.execute(sql_insert_personne, tuple(row))

In [ ]:
for index, row in df_oeuvre3.iterrows():
    cursor.execute(sql_insert_oeuvre, tuple(row))

In [ ]:
for index, row in df_activite3.iterrows():
    cursor.execute(sql_insert_activite, tuple(row))


In [ ]:
for index, row in df_poste3.iterrows():
    cursor.execute(sql_insert_poste, tuple(row))


In [ ]:
for index, row in df_mariage4.iterrows():
    cursor.execute(sql_insert_mariage, tuple(row))